In [1]:
import pandas as pd

In [2]:
finances = pd.read_pickle('IPL_finances.pkl')
results  = pd.read_pickle('IPL_results.pkl')

#Assume that the current 60crore rupees budget applied forever (rules for 2013 are not clear to me)
finances['Spend']=600000000.0-finances['Balance']
finances['Spend_crores']=finances['Spend']/10000000.0


In [3]:
#Get all teams ever
teams = results.index.values
print(teams)
finances.index

['Chennai Super Kings' 'Deccan Chargers' 'Delhi Capitals' 'Gujarat Lions'
 'Kings XI Punjab' 'Kochi Tuskers Kerala' 'Kolkata Knight Riders'
 'Mumbai Indians' 'Pune Warriors India' 'Rajasthan Royals'
 'Rising Pune Supergiant' 'Royal Challengers Bangalore'
 'Sunrisers Hyderabad']


MultiIndex(levels=[[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], [' Chennai Super Kings', 'Delhi', ' Gujarat Lions', ' Kings XI Punjab', ' Kolkata Knight Riders', ' Mumbai Indians', ' Pune Warriors India', ' Rajasthan Royals', ' Rising Pune Supergiant', ' Royal Challengers Bangalore', ' Sunrisers Hyderabad']],
           codes=[[7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 3, 4, 5, 7, 9, 10, 0, 1, 3, 4, 5, 7, 9, 10, 0, 1, 3, 4, 5, 7, 9, 10, 1, 2, 3, 4, 5, 8, 9, 10, 1, 2, 3, 4, 5, 8, 9, 10, 0, 1, 3, 4, 5, 7, 9, 10, 0, 1, 3, 4, 5, 7, 9, 10, 0, 1, 3, 4, 5, 6, 7, 9, 10]],
           names=['year', 'Team'])

In [4]:
#Total money spent ever in IPL (in crores)
sum(finances.Spend_crores)

3454.1495342

In [5]:
#Money spent by teams since 2013
finances.Spend_crores.groupby(level=[0]).sum()

year
2013    389.964534
2014    468.100000
2015    432.645000
2016    417.720000
2017    422.820000
2018    468.700000
2019    441.550000
2020    412.650000
Name: Spend_crores, dtype: float64

In [6]:

finances.Spend_crores.groupby(level=[1]).sum()

Team
 Chennai Super Kings            337.988869
Delhi                           441.148534
 Gujarat Lions                   99.150000
 Kings XI Punjab                383.005761
 Kolkata Knight Riders          428.396954
 Mumbai Indians                 462.150392
 Pune Warriors India             55.094807
 Rajasthan Royals               277.115046
 Rising Pune Supergiant         114.500000
 Royal Challengers Bangalore    456.709716
 Sunrisers Hyderabad            398.889454
Name: Spend_crores, dtype: float64